In [1]:
%defaultDatasource jdbc:h2:mem:db

# Criando as Tabelas

In [12]:
DROP Table IF EXISTS Filmes;
DROP Table IF EXISTS Series;
DROP Table IF EXISTS Netflix;
DROP Table IF EXISTS DisneyPlus;
DROP Table IF EXISTS Generos;
DROP Table IF EXISTS Paises;
DROP Table IF EXISTS Atores;
DROP Table IF EXISTS Diretores;
DROP TABLE IF EXISTS Palavras;

CREATE TABLE Filmes (
  Titulo VARCHAR(104) NOT NULL,
  Ano VARCHAR(4) NOT NULL,
  Duracao VARCHAR(7),
  Plataforma VARCHAR(15),
  ClassificacaoIndicativa VARCHAR(9),
  IMDb VARCHAR(6),
  RottenTomatoes VARCHAR(6),
  Enredo VARCHAR (3827),
  PRIMARY KEY(Titulo, Ano)
) AS SELECT
    Titulo,
    Ano,
    Duracao,
    Plataforma,
    "Classificacao Indicativa",
    IMDb,
    "Rotten Tomatoes",
    Enredo
FROM CSVREAD('filmesNew.csv');

CREATE TABLE Series (
  Titulo VARCHAR(93) NOT NULL,
  Ano VARCHAR(11) NOT NULL,
  Duracao VARCHAR(7),
  Temporadas VARCHAR (10),
  Plataforma VARCHAR(15),
  ClassificacaoIndicativa VARCHAR(9),
  IMDb VARCHAR(7),
  RottenTomatoes VARCHAR(7),
  Enredo VARCHAR (3457),
  PRIMARY KEY(Titulo, Ano)
) AS SELECT
    Titulo,
    Ano,
    Duracao,
    Temporadas,
    Plataforma,
    "Classificacao Indicativa",
    IMDb,
    "Rotten Tomatoes",
    Enredo
FROM CSVREAD('seriesNew.csv');

CREATE TABLE Netflix (
  Titulo VARCHAR(104) NOT NULL,
  Ano VARCHAR(4) NOT NULL,
  LancamentoNaPlataforma VARCHAR(19),
  PRIMARY KEY(Titulo, Ano)
) AS SELECT
    Titulo,
    Ano,
    "Lancamento na Plataforma"
FROM CSVREAD('netflix.csv');

CREATE TABLE DisneyPlus (
  Titulo VARCHAR(62) NOT NULL,
  Ano VARCHAR(11) NOT NULL,
  LancamentoNaPlataforma VARCHAR(17),
  PRIMARY KEY(Titulo, Ano)
) AS SELECT
    Titulo,
    Ano,
    "Lancamento na Plataforma"
FROM CSVREAD('disneyplus.csv');

CREATE TABLE Generos (
  Titulo VARCHAR(104) NOT NULL,
  Ano VARCHAR(11) NOT NULL,
  Genero VARCHAR(28) NOT NULL,
  PRIMARY KEY(Titulo, Ano, Genero)
) AS SELECT
    Titulo,
    Ano,
    Genero
FROM CSVREAD('generos.csv');

CREATE TABLE Paises (
  Titulo VARCHAR(104) NOT NULL,
  Ano VARCHAR(11) NOT NULL,
  Pais VARCHAR(30),
  PRIMARY KEY(Titulo, Ano, Pais)
) AS SELECT
    Titulo,
    Ano,
    Pais
FROM CSVREAD('paises.csv');

CREATE TABLE Atores (
  Titulo VARCHAR(104) NOT NULL,
  Ano VARCHAR(11) NOT NULL,
  Ator VARCHAR(100),
  PRIMARY KEY(Titulo, Ano, Ator)
) AS SELECT
    Titulo,
    Ano,
    Ator
FROM CSVREAD('atoresNew.csv');

CREATE TABLE Diretores (
  Titulo VARCHAR(104) NOT NULL,
  Ano VARCHAR(11) NOT NULL,
  Diretor VARCHAR(100),
  PRIMARY KEY(Titulo, Ano, Diretor)
) AS SELECT
    Titulo,
    Ano,
    Diretor
FROM CSVREAD('diretoresNew.csv');

CREATE TABLE Palavras (
  Palavra VARCHAR(23) NOT NULL
) AS SELECT
    Palavras
FROM CSVREAD('palavras.csv');

# Queries Faceis

## Quais filmes têm a melhor/pior avaliação?

In [ ]:
DROP VIEW IF EXISTS Movies;

CREATE VIEW Movies AS
SELECT SUBSTRING(IMDb, 1, 3) AS IMDb
FROM Filmes
WHERE IMDb NOT LIKE '%nan%'

* Melhor avaliação

In [ ]:
SELECT F.Titulo, F.IMDb
FROM Filmes F
WHERE SUBSTRING(F.IMDb, 1, 3) = (SELECT MAX(M.IMDb) FROM Movies M)

* Pior avaliação

In [ ]:
SELECT F.Titulo, F.IMDb
FROM Filmes F
WHERE SUBSTRING(F.IMDb, 1, 3) = (SELECT MIN(M.IMDb) FROM Movies M)

In [ ]:
DROP VIEW IF EXISTS Movies

## Qual o tempo entre a estreia de um filme e seu lançamento na plataforma?

* Netflix

In [ ]:
DROP VIEW IF EXISTS anoNetflix;

CREATE VIEW anoNetflix AS
SELECT CAST(Ano AS INT) AS Estreia, CAST(RIGHT(LancamentoNaPlataforma, 4) AS INT) AS LancamentoNetflix
FROM Netflix
WHERE Titulo IN (SELECT Titulo FROM Filmes)

In [ ]:
DROP VIEW IF EXISTS difNetflix;

CREATE VIEW difNetflix AS
SELECT LancamentoNetflix - Estreia AS TempoParaLancamento
FROM anoNetflix

In [ ]:
SELECT Titulo
FROM precaria
WHERE LancamentoNetflix - Estreia = -1

In [ ]:
SELECT TempoParaLancamento, COUNT(TempoParaLancamento) AS Qtd
FROM difNetflix
GROUP BY TempoParaLancamento
ORDER BY TempoParaLancamento

* Disney+

In [ ]:
DROP VIEW IF EXISTS anoDisney;

CREATE VIEW anoDisney AS
SELECT CAST(LEFT(Ano, 4) AS INT) AS Estreia, CAST(RIGHT(LancamentoNaPlataforma, 4) AS INT) AS LancamentoDisney
FROM DisneyPlus
WHERE Titulo IN (SELECT Titulo FROM Filmes)

In [ ]:
DROP VIEW IF EXISTS difDisney;

CREATE VIEW difDisney AS
SELECT LancamentoDisney - Estreia AS TempoParaLancamento
FROM anoDisney

In [ ]:
SELECT TempoParaLancamento, COUNT(TempoParaLancamento) AS Qtd
FROM difDisney
GROUP BY TempoParaLancamento
ORDER BY TempoParaLancamento

In [ ]:
DROP VIEW IF EXISTS difNetflix;
DROP VIEW IF EXISTS precaria;
DROP VIEW IF EXISTS anoNetflix;
DROP VIEW IF EXISTS difDisney;
DROP VIEW IF EXISTS anoDisney;

## Quantos filmes/séries foram feitos em cada país?

In [ ]:
SELECT Pais, COUNT(Pais) AS Qtd
FROM Paises
WHERE Pais NOT LIKE 'nan'
GROUP BY Pais
ORDER BY Qtd DESC

## Quais atores participaram de mais filmes?

In [ ]:
SELECT Ator, COUNT(Ator) AS Filmes
FROM Atores
WHERE Titulo IN (SELECT Titulo FROM Filmes) AND Ator NOT LIKE '%nan%'
GROUP BY Ator
ORDER BY Filmes DESC

## Quantos filmes/séries estão presentes em mais de uma plataforma?

* Filmes

In [ ]:
SELECT DISTINCT Plataforma FROM Filmes

In [ ]:
SELECT Titulo, Plataforma
FROM Filmes
WHERE Plataforma LIKE '%Netflix,Disney+%'

* Séries

In [ ]:
SELECT DISTINCT Plataforma FROM Series

In [ ]:
SELECT Titulo, Plataforma
FROM Series
WHERE Plataforma LIKE '%Netflix,Disney+%'

## Quais atores trabalharam em mídias de mais de uma plataforma?

* Filmes

In [ ]:
SELECT Ator
FROM Atores
WHERE Titulo IN (SELECT Titulo FROM Filmes WHERE Plataforma LIKE '%Netflix,Disney+%')
GROUP BY Ator

* Séries

In [ ]:
SELECT Ator
FROM Atores
WHERE Titulo IN (SELECT Titulo FROM Series WHERE Plataforma LIKE '%Netflix,Disney+%')
GROUP BY Ator

## Quais são os atores e diretores com mais participações em filmes +18?

In [ ]:
SELECT Ator, COUNT(Ator) AS Qtd
FROM Atores
WHERE Titulo IN (SELECT Titulo FROM Filmes WHERE ClassificacaoIndicativa IN ('18+', 'NR', 'R', 'NC-17', 'TV-MA'))
    AND Ator NOT LIKE '%nan%'
GROUP BY Ator
ORDER BY Qtd DESC

# Queries médias

## Quais gêneros são os mais frequentes em cada plataforma?

In [ ]:
--Cria tabelas com todas as obras de cada plataforma
DROP Table IF EXISTS allNetflix;
DROP Table IF EXISTS allDisneyPlus;

CREATE VIEW allNetflix AS
SELECT F.Titulo, F.Ano, F.Plataforma
    FROM Filmes F
    WHERE Plataforma = 'Netflix'
UNION
SELECT S.Titulo, S.Ano, S.Plataforma
    FROM Series S
    WHERE Plataforma = 'Netflix';
    
CREATE VIEW allDisneyPlus AS
SELECT F.Titulo, F.Ano, F.Plataforma
    FROM Filmes F
    WHERE Plataforma = 'Disney +'
UNION
SELECT S.Titulo, S.Ano, S.Plataforma
    FROM Series S
    WHERE Plataforma = 'Disney +'

In [ ]:
SELECT Genero, COUNT(Genero) AS Qtd
    FROM Generos G, allNetflix AN
    WHERE G.Titulo=AN.Titulo AND G.Ano=AN.Ano
    GROUP BY Genero
    HAVING COUNT(Genero)
    ORDER BY COUNT(Genero) DESC

In [ ]:
SELECT Genero, COUNT(Genero) AS Qtd
    FROM Generos G, allDisneyPlus AD
    WHERE G.Titulo=AD.Titulo AND G.Ano=AD.Ano
    GROUP BY Genero
    HAVING COUNT(Genero)
    ORDER BY COUNT(Genero) DESC

## Quais atores/diretores têm as melhores avaliações nos filmes em que participaram?

In [ ]:
DROP VIEW IF EXISTS AtoresAvaliacoes;
CREATE VIEW AtoresAvaliacoes AS
SELECT A.Ator, (CAST(SUBSTRING(F.IMDb, 1, LENGTH(F.IMDb) - 3) as DECIMAL(9,2)) * 10 + SUBSTRING(F.RottenTomatoes, 1, LENGTH(F.RottenTomatoes) - 4))/2 Avaliacao
        FROM Atores A, Filmes F
        WHERE A.Titulo = F.Titulo AND A.Ano = F.Ano AND F.IMDb != 'nan' AND F.IMDb != '' AND F.RottenTomatoes != 'nan' AND F.RottenTomatoes != ''
        ORDER BY A.Ator;

DROP VIEW IF EXISTS DiretoresAvaliacoes;
CREATE VIEW DiretoresAvaliacoes AS
SELECT D.Diretor, (CAST(SUBSTRING(F.IMDb, 1, LENGTH(F.IMDb) - 3) as DECIMAL(9,2)) * 10 + SUBSTRING(F.RottenTomatoes, 1, LENGTH(F.RottenTomatoes) - 4))/2 Avaliacao
        FROM Diretores D, Filmes F
        WHERE D.Titulo = F.Titulo AND D.Ano = F.Ano AND F.IMDb != 'nan' AND F.IMDb != '' AND F.RottenTomatoes != 'nan' AND F.RottenTomatoes != ''
        ORDER BY D.Diretor;

In [ ]:
SELECT AA.Ator, SUM(AA.Avaliacao)/COUNT(AA.Avaliacao) Media_Avaliacao
    FROM AtoresAvaliacoes AA
    GROUP BY AA.Ator
    ORDER BY Media_Avaliacao DESC

In [ ]:
SELECT DA.Diretor, SUM(DA.Avaliacao)/COUNT(DA.Avaliacao) Media_Avaliacao
    FROM DiretoresAvaliacoes DA
    GROUP BY DA.Diretor
    ORDER BY Media_Avaliacao DESC

## As plataformas Disney+ e Netflix concentram a disponibilização de conteúdo em alguma época do ano?

* Número de lançamentos em cada mês na plataforma Disney+

In [ ]:
DROP VIEW IF EXISTS mesDisneyPlus;

CREATE VIEW mesDisneyPlus AS
SELECT SUBSTRING(LancamentoNaPlataforma, 1, CHARINDEX(' ', LancamentoNaPlataforma)) AS Mes
FROM DisneyPlus

In [ ]:
SELECT D.Mes, COUNT(D.Mes) As Lancamentos
    FROM mesDisneyPlus D
    GROUP BY D.Mes
    ORDER BY Lancamentos DESC

* Número de lançamentos em cada mês na plataforma Netflix

In [ ]:
DROP VIEW IF EXISTS mesNetflix;

CREATE VIEW mesNetflix AS
SELECT SUBSTRING(LancamentoNaPlataforma, 1, CHARINDEX(' ', LancamentoNaPlataforma)) As Mes
FROM Netflix
WHERE LancamentoNaPlataforma NOT LIKE ''

In [ ]:
SELECT N.Mes, COUNT(N.Mes) As Lancamentos
    FROM mesNetflix N
    GROUP BY N.Mes
    ORDER BY Lancamentos DESC

In [ ]:
DROP VIEW mesDisneyPlus;
DROP VIEW mesNetflix;

## Qual a distribuição das avaliações das mídias?

* Distribuição das avaliações de filmes

In [ ]:
DROP VIEW IF EXISTS avFilmesString;

CREATE VIEW avFilmesString AS
SELECT SUBSTRING(F.IMDb, 1, 3) AS AvaliacaoString
FROM Filmes F
WHERE F.IMDb NOT LIKE 'nan/10'

In [ ]:
DROP VIEW IF EXISTS avFilmes;

CREATE VIEW avFilmes AS
SELECT CAST(AvaliacaoString AS FLOAT) AS Avaliacao
FROM avFilmesString

In [ ]:
SELECT
    CASE 
        WHEN Avaliacao between 0 and 0.4 then '0.0 - 0.4'
        WHEN Avaliacao between 0.5 and 0.9 then '0.5 - 0.9'
        WHEN Avaliacao between 1 and 1.4 then '1.0 - 1.4'
        WHEN Avaliacao between 1.5 and 1.9 then '1.5 - 1.9'
        WHEN Avaliacao between 2 and 2.4 then '2.0 - 2.4'
        WHEN Avaliacao between 2.5 and 2.9 then '2.5 - 2.9'
        WHEN Avaliacao between 3 and 3.4 then '3.0 - 3.4'
        WHEN Avaliacao between 3.5 and 3.9 then '3.5 - 3.9'
        WHEN Avaliacao between 4 and 4.4 then '4.0 - 4.4'
        WHEN Avaliacao between 4.5 and 4.9 then '4.5 - 4.9'
        WHEN Avaliacao between 5 and 5.4 then '5.0 - 5.4'
        WHEN Avaliacao between 5.5 and 5.9 then '5.0 - 5.9'
        WHEN Avaliacao between 6 and 6.4 then '6.0 - 6.4'
        WHEN Avaliacao between 6.5 and 6.9 then '6.5 - 6.9'
        WHEN Avaliacao between 7 and 7.4 then '7.0 - 7.4'
        WHEN Avaliacao between 7.5 and 7.9 then '7.5 - 7.9'
        WHEN Avaliacao between 8 and 8.4 then '8.0 - 8.4'
        WHEN Avaliacao between 8.5 and 8.9 then '8.5 - 8.9'
        WHEN Avaliacao between 9 and 9.4 then '9.0 - 9.4'
        ELSE '9.5 - 10'
    END AS IntervaloAvaliacao,
    COUNT(*) AS Qtd
FROM avFilmes
GROUP BY Avaliacao
ORDER BY Avaliacao

* Distribuição das avaliações de séries

In [ ]:
DROP VIEW IF EXISTS avSeriesStr;

CREATE VIEW avSeriesStr AS
SELECT SUBSTRING(S.IMDb, 1, 3) AS AvaliacaoString
FROM Series S
WHERE S.IMDb NOT LIKE 'nan/10'

In [ ]:
DROP VIEW IF EXISTS avSeries;

CREATE VIEW avSeries AS
SELECT CAST(AvaliacaoString AS FLOAT) AS Avaliacao
FROM avSeriesStr

In [ ]:
SELECT
    CASE 
        WHEN Avaliacao between 0 and 0.4 then '0.0 - 0.4'
        WHEN Avaliacao between 0.5 and 0.9 then '0.5 - 0.9'
        WHEN Avaliacao between 1 and 1.4 then '1.0 - 1.4'
        WHEN Avaliacao between 1.5 and 1.9 then '1.5 - 1.9'
        WHEN Avaliacao between 2 and 2.4 then '2.0 - 2.4'
        WHEN Avaliacao between 2.5 and 2.9 then '2.5 - 2.9'
        WHEN Avaliacao between 3 and 3.4 then '3.0 - 3.4'
        WHEN Avaliacao between 3.5 and 3.9 then '3.5 - 3.9'
        WHEN Avaliacao between 4 and 4.4 then '4.0 - 4.4'
        WHEN Avaliacao between 4.5 and 4.9 then '4.5 - 4.9'
        WHEN Avaliacao between 5 and 5.4 then '5.0 - 5.4'
        WHEN Avaliacao between 5.5 and 5.9 then '5.0 - 5.9'
        WHEN Avaliacao between 6 and 6.4 then '6.0 - 6.4'
        WHEN Avaliacao between 6.5 and 6.9 then '6.5 - 6.9'
        WHEN Avaliacao between 7 and 7.4 then '7.0 - 7.4'
        WHEN Avaliacao between 7.5 and 7.9 then '7.5 - 7.9'
        WHEN Avaliacao between 8 and 8.4 then '8.0 - 8.4'
        WHEN Avaliacao between 8.5 and 8.9 then '8.5 - 8.9'
        WHEN Avaliacao between 9 and 9.4 then '9.0 - 9.4'
        ELSE '9.5 - 10'
    END AS IntervaloAvaliacao,
    COUNT(*) AS Qtd
FROM avSeries
GROUP BY Avaliacao
ORDER BY Avaliacao

In [ ]:
DROP VIEW IF EXISTS avFilmes;
DROP VIEW IF EXISTS avFilmesString;
DROP VIEW IF EXISTS avSeries;
DROP VIEW IF EXISTS avSeriesStr;

## Taxa de classificação indicativa por ano, como o mercado lida com o envelhecimento do público?

* Classificação Indicativa dos filmes para cada ano

In [ ]:
DROP VIEW IF EXISTS ciFilmes;

CREATE VIEW ciFilmes AS
SELECT ClassificacaoIndicativa, Ano
FROM Filmes
WHERE ClassificacaoIndicativa NOT LIKE '' AND ClassificacaoIndicativa NOT LIKE '%min%'

In [ ]:
SELECT Ano, ClassificacaoIndicativa, COUNT(ClassificacaoIndicativa) AS Qtd
FROM ciFilmes
GROUP BY Ano, ClassificacaoIndicativa
ORDER BY Ano

* Classificação Indicativa dos séries para cada ano

In [ ]:
DROP VIEW IF EXISTS ciSeries;

CREATE VIEW ciSeries AS
SELECT ClassificacaoIndicativa, SUBSTRING(Ano, 1, 4) AS Ano
FROM Series
WHERE ClassificacaoIndicativa NOT LIKE ''

In [ ]:
SELECT Ano, ClassificacaoIndicativa, COUNT(ClassificacaoIndicativa) AS Qtd
FROM ciSeries
GROUP BY Ano, ClassificacaoIndicativa
ORDER BY Ano

In [ ]:
DROP VIEW IF EXISTS ciFilmes;
DROP VIEW IF EXISTS ciSeries;

## Comparação entre as avaliações Rotten Tomatoes e IMDb, com o objetivo de encontrar obras controversas

* Filmes

In [ ]:
SELECT F.Titulo, ABS(CAST(SUBSTRING(F.IMDb, 1, LENGTH(F.IMDb) - 3) as DECIMAL(9,2)) * 10 - SUBSTRING(F.RottenTomatoes, 1, LENGTH(F.RottenTomatoes) - 4)) DiferencaAvaliacao
    FROM Filmes F
    WHERE F.IMDb != 'nan' AND F.IMDb != '' AND F.RottenTomatoes != 'nan' AND F.RottenTomatoes != ''
    ORDER BY DiferencaAvaliacao DESC;

* Series

In [ ]:
SELECT S.Titulo, ABS(CAST(SUBSTRING(S.IMDb, 1, LENGTH(S.IMDb) - 3) as DECIMAL(9,2)) * 10 - SUBSTRING(S.RottenTomatoes, 1, LENGTH(S.RottenTomatoes) - 4)) DiferencaAvaliacao
    FROM Series S
    WHERE S.IMDb != 'nan' AND S.IMDb != '' AND S.RottenTomatoes != 'nan' AND S.RottenTomatoes != ''
    ORDER BY DiferencaAvaliacao DESC;

## Existe alguma relação entre popularidade e exclusividade dos serviços de streaming?

In [ ]:
--Cria tabelas com todas as obras de cada plataforma contendo a media da avaliação
DROP Table IF EXISTS allNetflix;
DROP Table IF EXISTS allDisneyPlus;
DROP Table IF EXISTS allOthers;


CREATE VIEW allNetflix AS
SELECT F.Titulo, F.Ano, F.Plataforma, (CAST(SUBSTRING(F.IMDB,1,LENGTH(F.IMDB)-3) AS DECIMAL(9,2))*10 + SUBSTRING(F.RottenTomatoes, 1, LENGTH(F.RottenTomatoes)-4))/2 Avaliacao
    FROM Filmes F
    WHERE F.Plataforma = 'Netflix' AND F.IMDB != 'nan' AND F.IMDB != '' AND F.IMDB != 'nan/10' AND F.RottenTomatoes != 'nan' AND F.RottenTomatoes != ''
UNION
SELECT S.Titulo, S.Ano, S.Plataforma, (CAST(SUBSTRING(S.IMDB,1,LENGTH(S.IMDB)-3) AS DECIMAL(9,2))*10 + SUBSTRING(S.RottenTomatoes, 1, LENGTH(S.RottenTomatoes)-4))/2 Avaliacao
    FROM Series S
    WHERE S.Plataforma = 'Netflix' AND S.IMDB != 'nan' AND S.IMDB != '' AND S.IMDB != 'nan/10' AND S.RottenTomatoes != 'nan' AND S.RottenTomatoes != '';

CREATE VIEW allDisneyPlus AS
SELECT F.Titulo, F.Ano, F.Plataforma, (CAST(SUBSTRING(F.IMDB,1,LENGTH(F.IMDB)-3) AS DECIMAL(9,2))*10 + SUBSTRING(F.RottenTomatoes, 1, LENGTH(F.RottenTomatoes)-4))/2 Avaliacao
    FROM Filmes F
    WHERE F.Plataforma = 'Disney +' AND F.IMDB != 'nan' AND F.IMDB != '' AND F.IMDB != 'nan/10' AND F.RottenTomatoes != 'nan' AND F.RottenTomatoes != ''
UNION
SELECT S.Titulo, S.Ano, S.Plataforma, (CAST(SUBSTRING(S.IMDB,1,LENGTH(S.IMDB)-3) AS DECIMAL(9,2))*10 + SUBSTRING(S.RottenTomatoes, 1, LENGTH(S.RottenTomatoes)-4))/2 Avaliacao
    FROM Series S
    WHERE S.Plataforma = 'Disney +' AND S.IMDB != 'nan' AND S.IMDB != '' AND S.IMDB != 'nan/10' AND S.RottenTomatoes != 'nan' AND S.RottenTomatoes != '';
    
CREATE VIEW allOthers AS
SELECT F.Titulo, F.Ano, F.Plataforma, (CAST(SUBSTRING(F.IMDB,1,LENGTH(F.IMDB)-3) AS DECIMAL(9,2))*10 + SUBSTRING(F.RottenTomatoes, 1, LENGTH(F.RottenTomatoes)-4))/2 Avaliacao
    FROM Filmes F
    WHERE F.Plataforma = 'Netflix,Disney+' OR F.Plataforma='Outra' AND F.IMDB != 'nan' AND F.IMDB != '' AND F.IMDB != 'nan/10' AND F.RottenTomatoes != 'nan' AND F.RottenTomatoes != ''
UNION
SELECT S.Titulo, S.Ano, S.Plataforma, (CAST(SUBSTRING(S.IMDB,1,LENGTH(S.IMDB)-3) AS DECIMAL(9,2))*10 + SUBSTRING(S.RottenTomatoes, 1, LENGTH(S.RottenTomatoes)-4))/2 Avaliacao
    FROM Series S
    WHERE S.Plataforma = 'Netflix,Disney+' OR S.Plataforma='Outra' AND S.IMDB != 'nan' AND S.IMDB != '' AND S.IMDB != 'nan/10' AND S.RottenTomatoes != 'nan' AND S.RottenTomatoes != '';

In [ ]:
SELECT AVG(Avaliacao) FROM allNetflix;
SELECT AVG(Avaliacao) FROM allDisneyPlus;
SELECT AVG(Avaliacao) FROM allOthers;

# Queries Difíceis

## Qual a palavra mais utilizada em títulos?

In [ ]:
SELECT Palavra, COUNT(Palavra) AS Qtd
FROM Palavras
WHERE LENGTH(Palavra) > 3
GROUP BY Palavra
ORDER BY Qtd DESC

In [ ]:
SELECT Palavra, COUNT(Palavra) AS Qtd
FROM Palavras
WHERE LENGTH(Palavra) <= 3
GROUP BY Palavra
ORDER BY Qtd DESC

## Existe alguma relação entre país e gênero?

* Dividido por Gêneros

In [ ]:
SELECT G.Genero, P.Pais, COUNT(G.Genero) AS Qtd
FROM Paises P, Generos G
WHERE P.Titulo = G.Titulo
GROUP BY Genero, Pais
ORDER BY Genero, Qtd DESC

* Dividido por Países

In [ ]:
SELECT P.Pais, G.Genero, COUNT(G.Genero) AS Qtd
FROM Paises P, Generos G
WHERE P.Titulo = G.Titulo
GROUP BY Pais, Genero
ORDER BY Pais, Qtd DESC

## Como os gêneros mais populares mudaram ao longo dos anos (tendências)?

In [ ]:
SELECT Genero, Ano, COUNT(Genero) AS Qtd
FROM Generos
GROUP BY Genero, Ano
ORDER BY Ano, Qtd DESC